# Hunter Lab Technical Assessment - Matthew Packham - 26.01.22

In [15]:
# dependencies
import json
import csv
import pandas as pd

## Data Exploration

I wanted to start by getting a general understanding of the structure of the .json file. 

I intially looked for the highest-level objects (`symbol`, `fundamentals` and `daily_trade_summary`) and dug into these by looking at their types. Since, depending on whether they are dictionary or list objects, they are dealt with differenently!
* `symbol` is just ticker!
* `Fundamentals` has the deepest structure with two levels of nested data. 
* `Daily_trade_summary` is just simply a list!

In [50]:
#data is relatively small 1.3Mb so can load to memory
filename = 'ibm_data.json'
with open(filename, 'r') as f:
        data = json.load(f)

In [71]:
###########  DATA EXPLORATION  ############

from pprint import pprint

# Higher level objects
print(f'Highest-Level objects:  {data.keys()}', end=2*'\n')


print("SYMBOL", end=2*'\n')
#symbol is just ticker - IBM
print(f'symbol:  {data["symbol"]}', end=2*'\n')


print("FUNDAMENTALS", end=2*'\n')
#Fundamentals
nested = data['fundamentals'].keys()
print(f'Fundamental keys:  {nested}', end=2*'\n')

#lets look into fundamentals to get an idea of the structure! 
for key in data['fundamentals'].keys():
    pprint(f"{key} has type {type(data['fundamentals'][key])}")
    for nested_key in data['fundamentals'][key].keys():
        if isinstance(data['fundamentals'][key][nested_key], str):
            continue
        else:
            try:
                pprint(f"-------->{nested_key} has type {type(data['fundamentals'][key][nested_key])}")
            except:
                continue
        

        
# daily_trade_summary
print('')
print("DAILY TRADE SUMMARY",  end=2*'\n')
print(f"daily_trade_summary has type {type(data['daily_trade_summary'])}")
dts = data['daily_trade_summary']
dts_df = pd.DataFrame(dts)
dts_df.head()

Highest-Level objects:  dict_keys(['symbol', 'fundamentals', 'daily_trade_summary'])

SYMBOL

symbol:  IBM

FUNDAMENTALS

Fundamental keys:  dict_keys(['overview', 'income_statements', 'balance_sheet_statements', 'cash_flow_statements', 'earning_stataments'])

"overview has type <class 'dict'>"
"income_statements has type <class 'dict'>"
"-------->annualReports has type <class 'list'>"
"-------->quarterlyReports has type <class 'list'>"
"balance_sheet_statements has type <class 'dict'>"
"-------->annualReports has type <class 'list'>"
"-------->quarterlyReports has type <class 'list'>"
"cash_flow_statements has type <class 'dict'>"
"-------->annualReports has type <class 'list'>"
"-------->quarterlyReports has type <class 'list'>"
"earning_stataments has type <class 'dict'>"
"-------->annualEarnings has type <class 'list'>"
"-------->quarterlyEarnings has type <class 'list'>"

DAILY TRADE SUMMARY

daily_trade_summary has type <class 'list'>


,date,1. open,2. high,3. low,4. close,5. volume
0,2021-11-12,120.0000,120.6400,118.7800,118.9600,5376966
1,2021-11-11,120.9000,121.7894,120.0800,120.2700,4643257
2,2021-11-10,121.0000,122.4300,119.9320,120.2200,6268961
3,2021-11-09,122.5600,122.9000,120.2600,120.8500,7195453
4,2021-11-08,123.9850,124.7800,123.5300,124.5400,5625275


## Creating CSV Files

Now I have gained an understanding of the inherant features of the .json file, I can now go about breaking it apart and converting it to csv files.

`write_to_csv` function converts a list to a csv.

Then I have written script which iterates through each of the keys and calls the `write_to_csv` function which converts them to csv. Since overview is the only dict obj, I just write this to .csv during the loop (rather than writing a seperate fun).

Note: I have written this script very specific to the .json file I recieved, however it is possible to make it more general so that it can take any format of .json file. I just need to add a few line regarding the keys, their types and how to deal with them (i.e. list or dict or empty keys etc...)

In [72]:
def write_to_csv(current_key_data, key_name):
    """ 
    function which converts list object to csv, by creating new csv file
    and writing the data to it!
    
    current_key_data: list | The data to convert to .csv
    Key_name: str | The name that will be used to save the csv file under
    """
    
    # now we will open a file for writing
    data_file = open(str(key_name) + '.csv', 'w')

    # create the csv writer object
    csv_writer = csv.writer(data_file)
    
    # Counter variable used for writing headers to the CSV file
    count = 0
    for report in current_key_data:
        if count == 0:

            # Writing headers of CSV file
            header = report.keys()
            csv_writer.writerow(header)
            count += 1

        # Writing data of CSV file
        csv_writer.writerow(report.values())
    #closing file
    data_file.close()
    
    return print(f'csv named {key_name} created')

In [75]:
################################################
                   #Script
#Iterates through each key and converts to .csv
################################################

filename = "ibm_data.json"
with open(filename, 'r') as f:
        data = json.load(f)
        
# loop through the data
for key in data.keys():
    #there are three 'top-level obj': symbol, fundamentals and daily_trade_summary
    
    #symbol
    if key == 'symbol':
        symbol = key
    
    #fundamentals
    elif key == 'fundamentals':
        for nested_key in data[key].keys():

            if nested_key == 'overview':
                
                current_key_data = data[key][nested_key]
                key_name = str(nested_key)
                
                # overview is a dictionary
                data_file = open('overview.csv','w')
                w = csv.writer(data_file)
                w.writerow(current_key_data.keys())
                w.writerow(current_key_data.values())
                data_file.close()
                print(f'csv named {key_name} created')    
                
            else:
                #'income_statements', 'balance_sheet_statements', 'cash_flow_statements', 'earning_stataments'
                for nest_nest_key in data[key][nested_key].keys():
                    #annualreports, quarterlyreports 
                    
                    current_key_data = data[key][nested_key][nest_nest_key]
                    str_nested, str_nest_nest = str(nested_key), str(nest_nest_key)
                    key_name = str_nested +'_'+str_nest_nest
                    write_to_csv(current_key_data, key_name)
    
    #daily_trade_summary
    else:
        current_key_data = data[key]
        key_name = key
        write_to_csv(current_key_data, key_name)
    

csv named overview created
csv named income_statements_annualReports created
csv named income_statements_quarterlyReports created
csv named balance_sheet_statements_annualReports created
csv named balance_sheet_statements_quarterlyReports created
csv named cash_flow_statements_annualReports created
csv named cash_flow_statements_quarterlyReports created
csv named earning_stataments_annualEarnings created
csv named earning_stataments_quarterlyEarnings created
csv named daily_trade_summary created


## Reading CSV Files

Here I print the first five rows of few csv examples!

#### Overview

In [82]:
overview_df = pd.read_csv('overview.csv')
overview_df.head()

,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
0,IBM,Common Stock,International Business Machines Corporation,International Business Machines Corporation (I...,51143,NYSE,USD,USA,TECHNOLOGY,COMPUTER & OFFICE EQUIPMENT,...,2.132,12.24,1.102,142.43,105.39,128.87,134.34,896320000,2021-12-10,2021-11-09


#### income_statements_annualReports 

In [81]:
income_statements_annualReports_df = pd.read_csv('income_statements_annualReports.csv')
income_statements_annualReports_df.head()

,fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,...,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome
0,2020-12-31,USD,35575000000,73620000000,38046000000,439000000,4609000000,23082000000,6333000000,30966000000,...,4227000000,2468000000,4726000000,-864000000,1288000000,5501000000,4850000000,6014000000,8482000000,5590000000
1,2019-12-31,USD,36488000000,77147000000,40659000000,591000000,9004000000,20604000000,5989000000,27484000000,...,4209000000,1850000000,10162000000,731000000,1344000000,9435000000,10324000000,11506000000,13356000000,9431000000
2,2018-12-31,USD,36936000000,79591000000,42655000000,42655000000,10838000000,19366000000,5379000000,26098000000,...,3127000000,1353000000,11347000000,2619000000,723000000,8723000000,8252000000,12070000000,13423000000,8728000000
3,2017-12-31,USD,36943000000,79139000000,42196000000,41703000000,8813000000,19680000000,5590000000,27414000000,...,3021000000,1520000000,11395000000,5642000000,615000000,5758000000,8559000000,12010000000,13530000000,5753000000
4,2016-12-31,USD,38516000000,79919000000,41403000000,40580000000,9715000000,20869000000,5726000000,28579000000,...,2837000000,1544000000,12321000000,449000000,630000000,11881000000,12081000000,12951000000,14495000000,11872000000


#### balance_sheet_statements_quarterlyReports

In [84]:
balance_sheet_statements_quarterlyReports_df = pd.read_csv('balance_sheet_statements_quarterlyReports.csv')
balance_sheet_statements_quarterlyReports_df.head()

,fiscalDateEnding,reportedCurrency,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,...,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding
0,2021-09-30,USD,144214000000,29967000000,7455000000,7455000000,1891000000,14669000000,112252000000,9138000000,...,6632000000,46926000000,83307000000,10460000000,15179000000,22228000000,169406000000,161747000000,57189000000,897097073
1,2021-06-30,USD,146814000000,30774000000,7350000000,7350000000,1807000000,15823000000,114030000000,9423000000,...,6406000000,48735000000,83835000000,11354000000,14741000000,21942000000,169404000000,162086000000,56912000000,895000000
2,2021-03-31,USD,148629000000,34038000000,10531000000,11131000000,1828000000,16067000000,112465000000,9452000000,...,5162000000,51206000000,83548000000,11670000000,14489000000,21389000000,169360000000,162218000000,56788000000,893630916
3,2020-12-31,USD,155971000000,39165000000,13212000000,13812000000,1839000000,18738000000,115864000000,10040000000,...,7053000000,54355000000,130178000000,13588000000,14897000000,20597000000,169339000000,162717000000,56556000000,892653424
4,2020-09-30,USD,154128000000,39845000000,14393000000,15593000000,1949000000,17870000000,112339000000,9958000000,...,9677000000,55129000000,100934000000,13017000000,15484000000,21208000000,169380000000,162806000000,56366000000,891400000


#### cash_flow_statements_annualReports 

In [85]:
cash_flow_statements_annualReports_df = pd.read_csv('cash_flow_statements_annualReports.csv')
cash_flow_statements_annualReports_df.head()

,fiscalDateEnding,reportedCurrency,operatingCashflow,paymentsForOperatingActivities,proceedsFromOperatingActivities,changeInOperatingLiabilities,changeInOperatingAssets,depreciationDepletionAndAmortization,capitalExpenditures,changeInReceivables,...,dividendPayoutCommonStock,dividendPayoutPreferredStock,proceedsFromIssuanceOfCommonStock,proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet,proceedsFromIssuanceOfPreferredStock,proceedsFromRepurchaseOfEquity,proceedsFromSaleOfTreasuryStock,changeInCashAndCashEquivalents,changeInExchangeRate,netIncome
0,2020-12-31,USD,18197000000,3406000000,None,138000000,-5088000000,6695000000,2618000000,-5297000000,...,5797000000,None,None,10504000000,None,-302000000,None,5448000000,None,5590000000
1,2019-12-31,USD,14770000000,3234000000,None,-503000000,-569000000,6059000000,2286000000,-502000000,...,5707000000,None,None,31825000000,None,-1361000000,None,-3124000000,None,9431000000
2,2018-12-31,USD,15247000000,1423000000,None,126000000,-879000000,4480000000,3395000000,-1006000000,...,5666000000,None,None,6891000000,None,-4443000000,None,-135000000,None,8728000000
3,2017-12-31,USD,16724000000,1208000000,None,47000000,-1315000000,4541000000,3229000000,-1297000000,...,5506000000,None,None,9643000000,None,-4165000000,None,4146000000,937000000,5753000000
4,2016-12-31,USD,17084000000,1158000000,None,197000000,-698000000,4957000000,3567000000,-712000000,...,5256000000,None,None,9132000000,None,-3298000000,None,140000000,-51000000,11872000000


#### daily_trade_summary 

In [16]:
daily_trade_summary_df = pd.read_csv('daily_trade_summary.csv')
daily_trade_summary_df.head()

,date,1. open,2. high,3. low,4. close,5. volume
0,2021-11-12,120.000,120.6400,118.780,118.96,5376966
1,2021-11-11,120.900,121.7894,120.080,120.27,4643257
2,2021-11-10,121.000,122.4300,119.932,120.22,6268961
3,2021-11-09,122.560,122.9000,120.260,120.85,7195453
4,2021-11-08,123.985,124.7800,123.530,124.54,5625275


## Creating mySQL db

Now that our data is in .csv, its very easy to create a mySQL db to store all IBM csv files. 

* Create an IBM db
* Create a table in db for each csv file



In [1]:
import mysql.connector

In [2]:
#example creation of a local db 
db = mysql.connector.connect(
    host='localhost',
    user='root',
    passwd='root'
)

In [3]:
# create a cursor obj
mycursor = db.cursor()

In [4]:
#create a new database

mycursor.execute('CREATE DATABASE IBM')

In [22]:
#connect to this specific db (IBM)

db = mysql.connector.connect(
    host='localhost',
    user='root',
    passwd='root',
    database='IBM' #connect to our specific db
)


In [23]:
# create a cursor obj
mycursor = db.cursor()

In [44]:
# Ive created an IBM db and now will create a table for each of the csv files
mycursor.execute("CREATE TABLE daily_trade_summary (date CHAR(50), open FLOAT, high FLOAT, low FLOAT, close FLOAT, volume int)")

In [45]:
for i,row in daily_trade_summary_df.iterrows():
    sql = "INSERT INTO IBM.daily_trade_summary VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))

In [48]:
#get a description of the atrributes of daily_trade_summary table 
mycursor.execute('DESCRIBE daily_trade_summary')
for x in mycursor:
    print(x)

('date', b'char(50)', 'YES', '', None, '')
('open', b'float', 'YES', '', None, '')
('high', b'float', 'YES', '', None, '')
('low', b'float', 'YES', '', None, '')
('close', b'float', 'YES', '', None, '')
('volume', b'int', 'YES', '', None, '')


In [49]:
# lets check it went through - print first 10 values
mycursor.execute('SELECT * FROM daily_trade_summary LIMIT 10') #selecting everything(*) from table called Person

for x in mycursor:
    print(x)

('2021-11-12', 120.0, 120.64, 118.78, 118.96, 5376966)
('2021-11-11', 120.9, 121.789, 120.08, 120.27, 4643257)
('2021-11-10', 121.0, 122.43, 119.932, 120.22, 6268961)
('2021-11-09', 122.56, 122.9, 120.26, 120.85, 7195453)
('2021-11-08', 123.985, 124.78, 123.53, 124.54, 5625275)
('2021-11-05', 121.43, 123.77, 121.43, 123.61, 6790478)
('2021-11-04', 123.05, 123.34, 119.9, 120.85, 7208736)
('2021-11-03', 126.23, 127.29, 125.68, 127.13, 5421406)
('2021-11-02', 126.3, 127.17, 124.91, 126.18, 4496393)
('2021-11-01', 125.05, 126.31, 123.838, 126.28, 5874767)


We can then reiterate this for each csv into a new table!